In [2]:
import py_stringsimjoin as ssj
from py_stringsimjoin.debugmatcher.debug_matcher import *
from py_stringsimjoin.labeler.labeler import *
from py_stringsimjoin.active_learner.active_learner import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import py_stringmatching as sm
from py_stringsimjoin.sampler.sample import *
from py_stringsimjoin.feature.autofeaturegen import *
from py_stringsimjoin.feature.extractfeatures import *
from py_stringsimjoin.utils.tokenizers import *

In [3]:
ws=sm.WhitespaceTokenizer(return_set=True)
qg2=sm.QgramTokenizer(qval=2,return_set=True)
qg3=sm.QgramTokenizer(qval=3,return_set=True)
num=NumericTokenizer(return_set=True)

In [4]:
ldf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/restaurants/fodors.csv')
rdf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/restaurants/zagats.csv')

In [5]:
ldf.head()

,id,name,addr,city,phone,type,class
0,534,'arnie morton\'s of chicago','435 s. la cienega blv.','los angeles',310/246-1501,american,0
1,535,'art\'s delicatessen','12224 ventura blvd.','studio city',818/762-1221,american,1
2,536,'hotel bel-air','701 stone canyon rd.','bel air',310/472-1211,californian,2
3,537,'cafe bizou','14016 ventura blvd.','sherman oaks',818/788-3536,french,3
4,538,campanile,'624 s. la brea ave.','los angeles',213/938-1447,american,4


In [6]:
rdf.head()

,id,name,addr,city,phone,type,class
0,1,'apple pan the','10801 w. pico blvd.','west la',310-475-3585,american,534
1,2,'asahi ramen','2027 sawtelle blvd.','west la',310-479-2231,'noodle shops',535
2,3,'baja fresh','3345 kimber dr.','westlake village',805-498-4049,mexican,536
3,4,'belvedere the','9882 little santa monica blvd.','beverly hills',310-788-2306,'pacific new wave',537
4,5,'benita\'s frites','1433 third st. promenade','santa monica',310-458-2889,'fast food',538


In [8]:
ldf['match_col']=ldf['name']+ ' ' +ldf['addr']+ ' '+ldf['city']+' '+ldf['phone']+' '+ldf['type']+' '+ldf['class'].astype(str)
rdf['match_col']=rdf['name']+ ' ' +rdf['addr']+ ' '+rdf['city']+' '+rdf['phone']+' '+rdf['type']+' '+rdf['class'].astype(str)

In [4]:
import re
#ldf['cln_title'] = ldf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
#rdf['cln_title'] = rdf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
l=[]
for idx, row in ldf.iterrows():
    s=re.sub(r'[^\w\s]',' ',str(row['title']).lower())
    if pd.notnull(row['authors']):
        s += re.sub(r'[^\w\s]',' ',str(row['authors']).lower())
    l.append(s)
r=[]
for idx, row in rdf.iterrows():
    s=re.sub(r'[^\w\s]',' ',str(row['title']).lower())
    if pd.notnull(row['authors']):
        s += re.sub(r'[^\w\s]',' ',str(row['authors']).lower())
    r.append(s)
ldf['cln_title'] = l
rdf['cln_title'] = r

In [9]:
seed=pd.read_csv('/scratch/resseed.csv')

In [10]:
c=sample_pairs(ldf,rdf,'id','id','match_col','match_col',1000,20,seed)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [11]:
len(c)

1004

In [12]:
labeled_c = label_table_using_gold(c, 'l_id', 'r_id', '/scratch/resgold.csv')
sum(labeled_c['label'])

19

In [15]:
ft=get_features(['JACCARD','COSINE','DICE','OVERLAP_COEFFICIENT'])
ft

,sim_measure_type,tokenizer,sim_function
feature_name,,,
jaccard_qg3,JACCARD,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138ef35550>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
jaccard_ws,JACCARD,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f138ef35610>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
jaccard_qg2,JACCARD,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138ef35650>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
cosine_qg3,COSINE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138ef35550>,<bound method Cosine.get_raw_score of <py_stringmatching.similarity_measure.cosine.Cosine object...
cosine_ws,COSINE,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f138ef35610>,<bound method Cosine.get_raw_score of <py_stringmatching.similarity_measure.cosine.Cosine object...
cosine_qg2,COSINE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138ef35650>,<bound method Cosine.get_raw_score of <py_stringmatching.similarity_measure.cosine.Cosine object...
dice_qg3,DICE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138ef35550>,<bound method Dice.get_raw_score of <py_stringmatching.similarity_measure.dice.Dice object at 0x...
dice_ws,DICE,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f138ef35610>,<bound method Dice.get_raw_score of <py_stringmatching.similarity_measure.dice.Dice object at 0x...
dice_qg2,DICE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138ef35650>,<bound method Dice.get_raw_score of <py_stringmatching.similarity_measure.dice.Dice object at 0x...


In [16]:
#ft.ix['jaccard_ws']['tokenizer'] = qg3
fvs = extract_feature_vecs(c, 'l_id','r_id',ldf,rdf,'id','id','match_col','match_col',ft,n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [17]:
len(fvs)

1004

In [18]:
from sklearn.ensemble import RandomForestClassifier
lr=RandomForestClassifier()
al=ActiveLearner(lr,20,20,'/scratch/resgold.csv',seed)

In [19]:
lp = al.learn(fvs, '_id', 'l_id','r_id')

     l_id r_id  jaccard_qg3  jaccard_ws  jaccard_qg2  cosine_qg3  cosine_ws  \
_id                                                                           
1000  534  219     0.700000    0.647059     0.776596    0.823824   0.785714   
1001  534  220     0.074830    0.000000     0.177966    0.140875   0.000000   
1002  535  220     0.651685    0.538462     0.720000    0.792220   0.700000   
1003  535  219     0.093168    0.043478     0.246032    0.171163   0.084515   

      cosine_qg2  dice_qg3   dice_ws  dice_qg2  overlap_coefficient_qg3  \
_id                                                                       
1000    0.874644  0.823529  0.785714  0.874251                 0.846154   
1001    0.306382  0.139241  0.000000  0.302158                 0.164179   
1002    0.841493  0.789116  0.700000  0.837209                 0.865672   
1003    0.396719  0.170455  0.083333  0.394904                 0.187500   

      overlap_coefficient_ws  overlap_coefficient_qg2  
_id               

py_stringsimjoin/active_learner/active_learner.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  to_be_labeled_pairs['label'] = labels


In [38]:
len(lr.estimators_

10

In [20]:
lp.reset_index(level=0, inplace=True)
sum(lp['label'])

19.0

In [21]:
ft1=get_features(['JACCARD','COSINE','DICE','OVERLAP_COEFFICIENT'])
ft1

,sim_measure_type,tokenizer,sim_function
feature_name,,,
jaccard_qg3,JACCARD,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138cd87690>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
jaccard_ws,JACCARD,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f138cd87490>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
jaccard_qg2,JACCARD,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138cd87750>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
cosine_qg3,COSINE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138cd87690>,<bound method Cosine.get_raw_score of <py_stringmatching.similarity_measure.cosine.Cosine object...
cosine_ws,COSINE,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f138cd87490>,<bound method Cosine.get_raw_score of <py_stringmatching.similarity_measure.cosine.Cosine object...
cosine_qg2,COSINE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138cd87750>,<bound method Cosine.get_raw_score of <py_stringmatching.similarity_measure.cosine.Cosine object...
dice_qg3,DICE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138cd87690>,<bound method Dice.get_raw_score of <py_stringmatching.similarity_measure.dice.Dice object at 0x...
dice_ws,DICE,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f138cd87490>,<bound method Dice.get_raw_score of <py_stringmatching.similarity_measure.dice.Dice object at 0x...
dice_qg2,DICE,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f138cd87750>,<bound method Dice.get_raw_score of <py_stringmatching.similarity_measure.dice.Dice object at 0x...


In [25]:
reload(ssj.feature.extractfeatures)
lp1=lp.copy()
del lp1['jaccard_ws']
ldf['id']=ldf['id'].astype(str)
rdf['id']=rdf['id'].astype(str)
fvs1 = extract_feature_vecs(lp1, 'l_id','r_id',ldf,rdf,'id','id','match_col','match_col',ft1,n_jobs=4)

0%      100%
[##########] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [26]:
len(set(fvs1.index.values))

42

In [27]:
fvs1['label'] = lp1['label']

In [28]:
from py_stringsimjoin.matcher.select_matcher import select_matcher
reload(ssj.matcher.select_matcher)
out=select_matcher(fvs1, 'label', ft1, metric='f1')
out['cv_stats']

,Name,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,dice_qg2,5,0.571429,0.400000,1.000000,0.800000,1.000000,0.754286
1,dice_qg3,5,0.666667,0.666667,1.000000,0.909091,1.000000,0.848485
2,cosine_qg2,5,0.571429,0.400000,1.000000,0.800000,1.000000,0.754286
3,cosine_qg3,5,0.800000,0.666667,1.000000,0.909091,1.000000,0.875152
4,overlap_coefficient_ws,5,0.800000,0.333333,1.000000,0.909091,0.750000,0.758485
5,jaccard_ws,5,0.800000,0.500000,0.750000,0.800000,0.333333,0.636667
6,jaccard_qg2,5,0.800000,1.000000,1.000000,0.909091,1.000000,0.941818
7,jaccard_qg3,5,1.000000,1.000000,1.000000,0.909091,1.000000,0.981818
8,overlap_coefficient_qg3,5,0.800000,0.666667,1.000000,0.909091,1.000000,0.875152
9,overlap_coefficient_qg2,5,0.800000,0.400000,1.000000,0.909091,1.000000,0.821818


In [33]:
from py_stringsimjoin.matcher.matcher_utils import split_train_test
splits = split_train_test(fvs1, 'label')
P =splits['train']
Q=splits['test']

In [34]:
type(out['selected_matcher'])

sklearn.linear_model.logistic.LogisticRegression

In [35]:
lr1=LogisticRegression()
dbg = debug_matcher(lr1, P, Q, 'jaccard_qg3', 'label', 'l_id', 'r_id', ldf, rdf, 'id', 'id')

0.428646368158


In [37]:
dbg['false_neg'][['l_match_col', 'r_match_col', 'jaccard_qg3']]

,l_match_col,r_match_col,jaccard_qg3
40,'restaurant katsu' '1972 n. hillhurst ave.' 'los angeles' 213/665-1891 asian 10,katsu '1972 hillhurst ave.' 'los feliz' 213-665-1891 japanese 10,0.390476


In [36]:
dbg['stats']

{'f1': 0.9411764705882353, 'precision': 1.0, 'recall': 0.8888888888888888}

In [22]:
-lr1.intercept_/lr1.coef_

array([[ 0.61488554]])

In [42]:
len(dbg['false_neg']), len(dbg['false_pos'])

(19, 7)

In [29]:
lr2=LogisticRegression()
lr2.fit(fvs1[['jaccard_qg3']].values, fvs1['label'].values)
-lr2.intercept_/lr2.coef_

array([[ 0.43902888]])

In [59]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(lp[['jaccard_ws']].values, lp['label'].values)
-lsvc.intercept_/lsvc.coef_

array([[ 0.56236397]])

In [65]:
what_if_matrix(lp, 'jaccard_ws', 'label', 0.57, 0.02, 10)

,threshold,precision,recall,f1
0,0.37,0.557377,0.988372,0.712788
1,0.39,0.599291,0.982558,0.744493
2,0.41,0.639847,0.970930,0.771363
3,0.43,0.670683,0.970930,0.793349
4,0.45,0.694561,0.965116,0.807786
5,0.47,0.723684,0.959302,0.825000
6,0.49,0.752336,0.936047,0.834197
7,0.51,0.791878,0.906977,0.845528
8,0.53,0.812834,0.883721,0.846797
9,0.55,0.808989,0.837209,0.822857


In [29]:
lp1=lp[lp['jaccard_ws'] >= 0.34]
len(lp1)

376

In [30]:
lp1.head()

,_id,l_id,r_id,jaccard_ws,label
0,100000,conf/vldb/ChakrabartiDAR97,8IR1NrQ4zm8J,0.784000,1
2,100002,conf/sigmod/LabrinidisR00,DKUouEkK_b4J,1.000000,1
24,37211,journals/sigmod/EnglertGH95,JRXuKm3E_foJ,1.000000,1
25,5067,journals/sigmod/NobleS95,rRvcD6HKhasJ,0.941176,1
26,61027,journals/tods/Chomicki03,pHuL-9kHWH0J,1.000000,1


In [51]:
cc=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.5777797,l_out_attrs=['year'], r_out_attrs=['year'],n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:39


In [53]:
len(cc)

5070

In [54]:
labeled_cc = label_table_using_gold(cc, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc['label'])


4626

In [55]:
4626/5070.0, 4626/5347.0

(0.9124260355029585, 0.8651580325416122)

In [56]:
cc1=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.34,l_out_attrs=['year'], r_out_attrs=['year'],n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:00


In [57]:
labeled_cc1 = label_table_using_gold(cc1, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc1['label'])

5270

In [30]:
cc2=ssj.jaccard_join(ldf,rdf,'id','id','match_col','match_col',qg3,0.43902888)


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [31]:
labeled_cc2 = label_table_using_gold(cc2, 'l_id', 'r_id', '/scratch/resgold.csv')
sum(labeled_cc2['label'])

101

In [32]:
len(cc2)

102

In [29]:
4752/5340.0, 4752/5347.0

(0.8898876404494382, 0.8887226482139517)